In [1]:
from docplex.mp.model import Model
import pandas as pd
import numpy as np
np.seterr(all="ignore")
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

def euclid_dist (x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

#input data from .csv file
wh_coords = pd.read_csv("./coordinates.csv")
x_coord=wh_coords['x'].values
y_coord=wh_coords['y'].values
product_data=pd.read_csv("./product_data.csv", index_col=0)
Q=product_data['TOT_MP'].values
W=product_data['VOL_DEPLETING'].values
#MP=product_data['MP'].values 
VOL = product_data['VOL'].values
LC = product_data['#LC'].values
dis = []
for i in range(len(wh_coords)):
    dis.append(euclid_dist(424,26.155,x_coord[i],y_coord[i]))

#input data from context
Class=3 
L=len(dis)
P=len(product_data)
C = 2.5 #Slot max capacity
Z = 10
Zone_1=range(1,12)
Zone_2=range(13,49)
Zone_3=range(50,172)
Zone_4=range(173,208)
Zone_5=range(209,347)
Zone_6=range(348,359)
Zone_7=range(360,400)
Zone_8=range(401,679)
Zone_9=range(680,777)
Zone_10=range(778,827)
Stock_Zones=[Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6, Zone_7, Zone_8, Zone_9, Zone_10]

#Apc Matrix
A=np.zeros((P,Class),int)   
product_data['Class']='C' 
for i in range(len(product_data)):
    if W[i]>=15: 
        product_data['Class'][i] ='A' 
        A[[i],0] = 1 
    elif W[i]>=10 and W[i]<15:
        product_data['Class'][i]='B'
        A[[i],1]=1 
    else:
        product_data['Class'][i]='C'
        A[[i],2]=1 

restricted = []
for i in range(len(product_data)):
    if product_data['Class'][i] =='A': 
        restricted.append(i)

mdl = Model(name= 'WH_REVAMP') 

# Decision Variables
x = mdl.binary_var_matrix(L, P, name='X')  # 1 if product p is assigned to slot l, 0 otherwise
y = mdl.integer_var_matrix(L, P, name='Y')  # Number of load carriers of product p in slot l

#CONSTRAINTS:

# 1. At most one product type per location
for l in range(L):
    mdl.add_constraint(mdl.sum(x[l, p] for p in range(P)) <= 1)

# 2. Slot capacity
for l in range(L):
    for p in range(P):
        mdl.add_constraint(mdl.sum(VOL[p] * y[l, p]) <= C * x[l, p])

# 3. No empty slots in a row if it ends with an occupied slot
for l in range(1, L - 1):  # Iterate from the second slot to the second-to-last slot
    if l % 12 != 0 and l % 12 != 11:  # Not the first or last slot in a row
        mdl.add_constraint(
            mdl.sum(x[l, p] for p in range(P)) >= mdl.sum(x[l - 1, p] for p in range(P)) + mdl.sum(x[l + 1, p] for p in range(P)) - 1
        )

# 4. Linking x and y variables
for p in range(P):
    mdl.add_constraint(mdl.sum(y[l, p] for l in range(L)) == LC[p])

# Weights for product classes (Adjust these as needed)
weight_A = 1
weight_B = 2
weight_C = 3

# OBJ FUNCTION
mdl.minimize(mdl.sum(
    mdl.sum(dis[l] * x[l, p] * (3*A[p, 0] + 2*A[p, 1] + 1*A[p, 2]) for p in range(P)) 
    for l in range(L)
))

#print the implementation and solution of the objective function and the constraints

#print(mdl.export_to_string())

# Set CPLEX parameters for detailed logging
mdl.parameters.mip.display = 2  # Display every node and integer solution
mdl.parameters.parallel = -1 # oportunistic
mdl.parameters.workmem = 16000 # 10 GB 

sol=mdl.solve(log_output=True, clean_before_solve=True, TimeLimits=20, LogPeriod=100, LogVerbosity='Verbose')
sol.display()

#save the solution in a list
x_list = []
for l in range(L):
    tmp = False
    for p in range(P):
        if sol.get_value(x[l, p]) == 1:
            tmp = True 
            x_list.append([l+1, p+1])
    if not tmp:
        x_list.append([l+1, -1])

print(x_list)

#save data frame product_data into a .csv
product_data.to_csv("./products_data(output).csv", index=True, header=True)

# Visualization
fig = go.Figure()

# Add empty slots first
empty_slots = [l for l, p in x_list if p == -1]
empty_x = x_coord[np.array(empty_slots)-1]
empty_y = y_coord[np.array(empty_slots)-1]
empty_text = [f"Slot: {l}<br>Empty" for l in empty_slots]  # Corrected

fig.add_trace(go.Scatter(
    x=empty_x,
    y=empty_y,
    mode='markers',
    marker=dict(color='green', size=10),
    text=empty_text,
    hoverinfo='text',
    name='Empty'
))

# Add slots for each product class with corresponding colors
for product_class, color in zip(['A', 'B', 'C'], ['blue', 'yellow', 'purple']):
    occupied_slots = [l for l, p in x_list if p != -1 and product_data['Class'][p-1] == product_class]
    occupied_x = x_coord[np.array(occupied_slots)-1]
    occupied_y = y_coord[np.array(occupied_slots)-1]
    
    # Include product type index in hover text
    occupied_text = [
        f"Slot: {l}<br>Class: {product_data['Class'][p-1]}<br>#LC: {int(sol.get_value(y[l-1, p-1]))}<br>Product Index: {p}" 
        for l, p in x_list if p != -1 and product_data['Class'][p-1] == product_class
    ]

    fig.add_trace(go.Scatter(
        x=occupied_x,
        y=occupied_y,
        mode='markers',
        marker=dict(color=color, size=10),
        text=occupied_text,
        hoverinfo='text',
        name=f'Product {product_class}'
    ))


# Update layout
fig.update_layout(
    title='Warehouse Slot Occupancy',
    xaxis_title='X Coordinate',
    yaxis_title='Y Coordinate',
    hovermode='closest',
    showlegend=True
)

fig.show()

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Parallel                                -1
CPXPARAM_WorkMem                                 16000
Tried aggregator 2 times.
MIP Presolve modified 74430 coefficients.
Aggregator did 414 substitutions.
Reduced MIP has 38362 rows, 74016 columns, and 241047 nonzeros.
Reduced MIP has 39282 binaries, 34734 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.34 sec. (523.08 ticks)
Probing time = 0.14 sec. (25.38 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 38362 rows, 74016 columns, and 241047 nonzeros.
Reduced MIP has 39282 binaries, 34734 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.20 sec. (147.51 ticks)
Probing time = 0.08 sec. (24.23 ticks)
Clique table members: 95912.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: opportunistic, using up to 8 threads.
Root relaxation solution time = 1.